## Fake News Classifier Using LSTM

In this project, I would try to implement LSTM (Long short-term memory) to classify fake news based on the `title` of the news. LSTM is an artificial neural network used in the field of deep learning and is heavily used in NLP based tasks.

The dataset has been taken from kaggle: https://www.kaggle.com/c/fake-news/data#

In [ ]:
# Importing Libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dropout

In [ ]:
# Checking tensorflow version
tf.__version__

In [ ]:
# Read the dataset
df=pd.read_csv('train.csv')
df.head()

In [ ]:
# Drop Nan Values
df=df.dropna()
df.reset_index(inplace=True, drop = True)

In [ ]:
# Get the Independent Features
X=df.drop('label',axis=1)

In [ ]:
# Get the Dependent features
y=df['label']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Vocabulary size (size of the vocabulary dictionary)
voc_size=5000

### One-hot Representation

The first step here would be to convert the collection of words into a one-hot representation.
But before that, we need to make sure the data is pre-processed.

In [ ]:
# creating a copy of X
messages=X.copy()

In [ ]:
messages['title'][0]

In [ ]:
nltk.download('stopwords')

In [ ]:
# Dataset Preprocessing

ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # removing unnecessary characters
    review = review.lower() # converting to lower case
    review = review.split() # removing whitespaces
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # stemming the words
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[0]

In [ ]:
# Finally, converting to one-hot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]

### Embedding Representation

In [ ]:
sent_length=20 # max length of sentence we are allowing
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length) # pre-padding used here
print(embedded_docs)

In [ ]:
embedded_docs[0]

### Building the model

In [ ]:
embedding_vector_features=40 # Number of features in the Feature representation
model=Sequential() # Building a sequential model
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # Adding an embedding layer
model.add(LSTM(100)) # Adding an LSTM layer with 100 neurons
model.add(Dense(1,activation='sigmoid')) # Adding a dense layer with sigmoid activation function
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) # Compiling the model
print(model.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=42)

### Model Training

In [ ]:
# Model training with 10 epochs (Will take some time!)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

### Performance Metrics And Accuracy

In [ ]:
y_pred=model.predict_classes(X_test)

accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

So we got a pretty good accuracy score of 0.89 which is much better than the Logistic regression score of 0.81 which I previously trained. To further improve the score, I tried adding two dropout layers to the sequential model as shown below.

### Adding Dropout 

In [ ]:
# Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3)) # Dropout of 0.3
model.add(LSTM(100))
model.add(Dropout(0.3)) # Dropout of 0.3
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Model training with 10 epochs (Will take some time!)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [ ]:
y_pred=model.predict_classes(X_test)

accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

## Summary

We were able to classify the fake news titles with a pretty good accuracy of 0.90 on the test data by using LSTM with additional dropout layers. We could also try playing around with some hyperparameters to improve the model accuracy such as vocab size, embedding vector features, etc. 